##**Importing Necessary Libraries**

In [ ]:
import pathlib
import pandas as pd 
import numpy as np
import nltk
import re
!pip install -q -U "tensorflow-text==2.8.*"
import tensorflow as tf
import tensorflow_text as text
import functools
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 4.9 MB 7.4 MB/s 
     |████████████████████████████████| 497.9 MB 34 kB/s 
     |████████████████████████████████| 5.8 MB 52.3 MB/s 
     |████████████████████████████████| 1.4 MB 69.7 MB/s 
     |████████████████████████████████| 462 kB 68.9 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

##**Importing Train File**

In [ ]:
df=pd.read_csv(r'/content/train.csv')

##**Preprocessing the Tweets**

In [ ]:
# Removing the Tags
df.Tweet = df.Tweet.str.replace(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+)", "").str.strip()
# Removing new line occurences
df.Tweet = df.Tweet.str.replace(r"\n","", regex = False).str.strip()
# Removing &s and following characters
df.Tweet = df.Tweet.str.replace(r"&[A-Za-z0-9]","", regex = False).str.strip()
# Converting everything to Lowercase
df.Tweet = df.Tweet.str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 6.4 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=9a7bbdcfdae2f9acb5c2e6ef35cc4ddc4a32590b86a4fb3a871bad5fd8eb1a6d
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [ ]:
import emoji
# Converting Emojis to their respective word expressions
df.Tweet = df.Tweet.apply(lambda x: emoji.demojize(x))

In [ ]:
def punct_remover(strng):
#     # punctuation marks to be completely removed
    clean_strng = re.sub(r'[?|!|\'|"|#]', r'', strng)
#     # punctuation marks to be replaced with space
    clean_strng = re.sub(r'[.|,|)|(|\|/]', r' ', clean_strng)
#     # replace multi-space with single space
    clean_strng = re.sub(r' +', r' ', clean_strng)

    return clean_strng

df.Tweet = df.Tweet.apply(punct_remover)

##**Result after Preprocessing**

In [ ]:
X_train = df.iloc[:,:]
X_train

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,2017-En-21441,“worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1,0
1,2017-En-31535,whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0,0
2,2017-En-21068,it also helps that the majority of nfl coachin...,1,0,1,0,1,0,1,0,0,0,0,0
3,2017-En-31436,accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0,0
4,2017-En-22195,my roommate: its okay that we cant spell becau...,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7719,2018-En-01993,if this didnt make me so angry id be laughing ...,1,0,1,0,0,0,0,0,0,0,0,0
7720,2018-En-01784,excited to watch stateoforigin tonight come on...,0,0,0,0,1,0,1,0,0,0,0,0
7721,2018-En-04047,blah blah blah kyrie it etc leaving boston is ...,1,0,1,0,0,0,0,0,1,0,0,0
7722,2018-En-03041,thingsivelearned the wise shepherd never trust...,0,0,0,0,0,0,0,0,0,0,0,1


##**Creating Word Embeddings (128 dimensional)**

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(X_train.Tweet)

<tf.Tensor: shape=(7724, 128), dtype=float32, numpy=
array([[ 0.33636957,  0.00701734, -0.10720108, ..., -0.01749418,
        -0.1550162 ,  0.1251124 ],
       [ 0.336427  ,  0.12187776,  0.18978862, ..., -0.16236518,
        -0.03249783,  0.20863476],
       [ 0.488294  ,  0.00090875, -0.04779435, ..., -0.06775768,
         0.08352137,  0.00292015],
       ...,
       [ 0.25731686,  0.11340199,  0.11283287, ..., -0.03014526,
         0.10251772, -0.04829827],
       [ 0.23004037,  0.05500153, -0.1474879 , ..., -0.21279554,
         0.04359892, -0.03445235],
       [ 0.56669146,  0.08317491,  0.27688044, ..., -0.10052701,
         0.02496464, -0.00662637]], dtype=float32)>

##**Defining a 3 layer densely connected Neural Network and an output layer of 12 dimensions**

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(12, activation= 'relu'))
model.add(tf.keras.layers.Dense(12, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'relu'))
model.add(tf.keras.layers.Dense(12, activation = 'sigmoid'))

model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f98d6415d10>>

##**Model Training**

In [ ]:
model.compile(Adam(learning_rate = 0.01), loss='binary_crossentropy', metrics=['accuracy'])
Model = model.fit(X_train.Tweet, X_train.iloc[:,2:], epochs=20, batch_size = 32)

##**Evaluating on test data**

In [ ]:
df_test=pd.read_csv(r'/content/test.csv')
df_test.head(3)

,ID,Tweet
0,test_record_1,i could tell that my white clo this review was...
1,test_record_2,bissell big green clean machine t love love lo...
2,test_record_3,i was looking for this


In [ ]:
Y_hat = model.predict(df_test.Tweet)

In [ ]:
Y_hat = map(lambda x: np.greater_equal(x,[0.45]).astype(int),Y_hat)

In [ ]:
labels = df.columns[2:]
labels

Index(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism',
       'pessimism', 'sadness', 'surprise', 'trust', 'neutral'],
      dtype='object')

In [ ]:
Y_hat = pd.DataFrame(Y_hat, columns = labels)

In [ ]:
Y_hat

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,1,0,1,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,1,0
4,0,1,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,1,0,0,0,0,1,0,0,0,0
396,1,0,1,0,0,0,0,0,0,0,0,0
397,0,0,1,0,0,0,0,0,0,1,0,0
398,1,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
Y_hat[Y_hat['neutral']==1]

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
39,0,0,0,0,0,0,0,0,0,0,0,1
103,0,0,0,0,0,0,0,0,0,0,0,1
384,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
result = pd.concat([df_test.ID,df_test.Tweet,Y_hat], axis = 1)

In [ ]:
result

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,test_record_1,i could tell that my white clo this review was...,1,0,1,0,1,0,0,0,0,0,0,0
1,test_record_2,bissell big green clean machine t love love lo...,0,0,0,0,1,1,1,0,0,0,0,0
2,test_record_3,i was looking for this,1,1,0,0,0,0,0,0,0,0,0,0
3,test_record_4,bissell spot clean professional portable carpe...,0,0,0,0,1,0,1,0,0,0,1,0
4,test_record_5,bissell automate cordless rechargeable hand va...,0,1,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,test_record_396,fauciliedpeopledied wrong buck stops at potus ...,0,0,1,0,0,0,0,1,0,0,0,0
396,test_record_397,would go by easyjet jet wizzair b a lufthansa ...,1,0,1,0,0,0,0,0,0,0,0,0
397,test_record_398,rt over dead day zero empathy from trumpgop tr...,0,0,1,0,0,0,0,0,0,1,0,0
398,test_record_399,all offerup letgo craigslist miles ebay amazon...,1,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
result.to_csv('submission1.csv')